In [1]:
n_rays = 50
case = "_Ex_7_2"

In [2]:
params = {
    "initialization": {
        "x_init": [2.5, 2.5],
        "y_init": [1.0, 1.5]
    },
    "exact":{
        "learning_rate": 0.009,
        "max_iters": 20000,
        "tol": 1e-5
    },
    "phase1": {
        "gamma": 12,
        "expo_gamma": 0.01,
        "max_iter": 10000,
        "tol": 1e-4
    },
    "phase2": {
        "expo_alpha": 0.45,
        "expo_beta": 0.4,
        "expo_lambda": 0.55,
        "init_params": 1.0,
        "max_iter": 2000,
        "mu": 1e-4,
        "verbose": False,
        "expo_gamma": 0.5
    }
}


In [3]:
import autograd.numpy as np
import torch

def set_seed(seed=702):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)

set_seed(702)

# Gen rays

In [4]:
import autograd.numpy as np
import math

In [5]:
def generate_chebyshev_rays(num_rays=10):
    rays = []
    angles = np.linspace(0, np.pi/2, num_rays)
    for theta in angles:
        r1, r2 = math.cos(theta), math.sin(theta)
        if abs(r1) < 1e-9: r1 = 0.0
        if abs(r2) < 1e-9: r2 = 0.0
        rays.append([r1, r2])       
    return np.array(rays)

In [6]:
test_rays = generate_chebyshev_rays(n_rays)

# Giải chính xác bằng phương pháp chiếu

## Bài toán

In [7]:
from scipy.optimize import Bounds

In [8]:
# ==========================================
# HÀM MỤC TIÊU
# ==========================================
def f1(x):    return (x[0]**2 + x[1]**2)/50
def f2(x):    return ((x[0] - 5)**2 + (x[1] - 5)**2)/50
def f(x):    return np.array([
    (x[0]**2 + x[1]**2)/50,
    ((x[0] - 5)**2 + (x[1] - 5)**2)/50])
def jacobian_f(x):
    return np.array([
        [x[0]/25.0, x[1]/25.0],
        [(x[0] - 5)/25.0, (x[1] - 5)/25.0]
    ])
# --- Hàm mục tiêu Weighted Chebyshev ---
def S_weighted_chebyshev(x, r, z):
    vals = f(x)
    weighted_diffs = r * np.abs(vals - z)
    return np.max(weighted_diffs)

def get_subgradient(x, r, z):
    vals = f(x)
    diffs = vals - z
    weighted_abs_diffs = r * np.abs(diffs)
    k = np.argmax(weighted_abs_diffs)
    
    J = jacobian_f(x)
    gradient_fk = J[k] 
    
    sign = 1.0 if diffs[k] >= 0 else -1.0
    grad = r[k] * sign * gradient_fk
    return grad
# ==========================================
# CÁC RÀNG BUỘC & PHÉP CHIẾU
# ==========================================
# Ràng buộc tập X
bounds_x = Bounds([0,0],[5, 5])
def constraints_X(x):
    # Trả về mảng các giá trị >= 0
    return np.array([
        x[0],           # x0 >= 0
        5.0 - x[0],     # x0 <= 5
        x[1],           # x1 >= 0
        5.0 - x[1]      # x1 <= 5
    ])

# --- Các ràng buộc tập Q ---
def q1(y): return 0.2**2 - (y[0] - 0.4)**2 - (y[1] - 0.4)**2
def q_plus(y):
    center = 0.4
    radius_sq = 0.2**2  
    dx = np.maximum(0, y[0] - center)
    dy = np.maximum(0, y[1] - center)
    return radius_sq - (dx**2 + dy**2)

dim_x = 2
dim_y = 2

## Phép chiếu lên X'

In [9]:
from Problem import Problem
from project import Projection
from scipy.optimize import minimize

In [10]:
cons_C = ()
cons_Q = ({'type': 'ineq', 'fun' : q1,},)
cons_Qplus = ({'type': 'ineq', 'fun': q_plus},)
# Setup Projections
proj_C_handler = Projection(cons=cons_C, bounds=bounds_x, dim=dim_x, proj_type='euclid')
proj_Q_handler = Projection(cons=cons_Q, bounds=None, dim=dim_y, proj_type='qplus')
# Setup Problem
prob = Problem(
    f=[f], jac_f=[jacobian_f],
    dim_x=dim_x, dim_y=dim_y,
    proj_C=proj_C_handler.project,
    proj_Qplus=proj_Q_handler.project
)

In [11]:
def project_step(x_temp, y_temp, dim_x, dim_y):
    """
    Chiếu điểm (x, y) vào tập:
    { (x, y) | x in X, y in Q, f(x) <= y }
    """
    z_start = np.concatenate([x_temp, y_temp])
    
    def proj_obj(z): 
        return 0.5 * np.sum((z - z_start)**2)
    
    def proj_jac(z): 
        return z - z_start

    # Danh sách ràng buộc: scipy yêu cầu dạng 'ineq' (tức là >= 0)
    cons = [
        # Ràng buộc x \in X
        {'type': 'ineq', 'fun': lambda z: constraints_X(z[:dim_x])},
        # Ràng buộc y \in Q
        {'type': 'ineq', 'fun': lambda z: q1(z[dim_x:])},
        # Ràng buộc f(x) <= y  <=>  y - f(x) >= 0
        {'type': 'ineq', 'fun': lambda z: z[dim_x:] - f(z[:dim_x])}
    ]

    res = minimize(
        fun=proj_obj, x0=z_start, jac=proj_jac,
        constraints=cons, method='SLSQP',
        options={'disp': False}
    )
    
    if not res.success:
        return x_temp, y_temp
    return res.x[:dim_x], res.x[dim_x:]

## Core

In [12]:
def solve_weighted_chebyshev(r_pref, z_ref, params , dim_x=1, dim_y=2):
    x = params["initialization"]["x_init"]    
    y = params["initialization"]["y_init"]
    learning_rate = params["exact"]["learning_rate"]
    max_iters = params["exact"]["max_iters"]
    tol = params["exact"]["tol"]
    prev_obj_val = float('inf')
    converge = False
    
    print(f"r={r_pref}")
    print(f"{'Iter':<5} | {'x':<10} | {'y vector':<15} | {'Objective':<10} | {'Active Idx'}")
    for k in range(max_iters):
        # --- BƯỚC 1: Tính Subgradient ---
        grad_x = get_subgradient(x, r_pref, z_ref)
        
        # --- BƯỚC 2: Cập nhật biến x ---
        x_temp = x - learning_rate * grad_x
        y_temp = y 
        
        # --- BƯỚC 3: Phép chiếu (Projection) ---
        x_next, y_next = project_step(x_temp, y_temp, dim_x, dim_y)
        
        # --- Tính giá trị hàm mục tiêu hiện tại ---
        obj_val = S_weighted_chebyshev(x_next, r_pref, z_ref)
        
        # --- Logging ---
        diffs = r_pref * np.abs(f(x_next) - z_ref)
        active_idx = np.argmax(diffs)
        
        if k % 2000 == 0:
            print(f"{k:<5} | {x_next[0]:.4f}     | [{y_next[0]:.2f}, {y_next[1]:.2f}] | {obj_val:.6f}     | f_{active_idx+1}")

        # --- BƯỚC 4: Kiểm tra điều kiện dừng ---
        loss_change = abs(prev_obj_val - obj_val)
        
        if loss_change < tol:
            print(f"--> Hội tụ tại bước {k}!\n")
            converge = True
            break
            
        prev_obj_val = obj_val
        x = x_next
        y = y_next

    return x, y, converge

## Tìm điểm tham chiếu

In [13]:
from Phase1 import solve_CQ_feasible
from Phase2_1_obj import optim_Universal

In [14]:
def relevant_point(prob, params):
    x_feasible, _, _, _ = solve_CQ_feasible(
        prob.objective_func, prob.jacobian, prob.proj_C, prob.proj_Qplus,
        x0=params['initialization']['x_init'],
        gamma=params['phase1']['gamma'] , max_iter=params['phase1']['max_iter']
    )

    z_vals = []
    nadir_vals = []

    for dim in range(2):
        # Tìm MIN
        x_min, _ = optim_Universal(prob, x_feasible, target_dim=dim, mode="min", max_iter=500, mu=0.001)
        val_min = prob.objective_func(x_min)[dim]
        z_vals.append(val_min)

        # Tìm MAX 
        x_max, _ = optim_Universal(prob, x_feasible, target_dim=dim, mode="max", max_iter=500, mu=0.001)
        val_max = prob.objective_func(x_max)[dim]
        nadir_vals.append(val_max)
        print(f"   Dim {dim}: Min={val_min:.4f}, Max={val_max:.4f}")
    z_star = np.array(z_vals)
    ref_point = np.array(nadir_vals) + np.abs(np.array(nadir_vals)) * 0.1 + 0.5 
    print(f"-> Z* (Ideal): {z_star}")
    print(f"-> Ref Point (HV): {ref_point}")
    print(f"-> x (Feasible): {x_feasible}")
    return z_star, ref_point, x_feasible

## Giải chính xác

In [15]:
z_star, ref_point, x_feasible = relevant_point(prob, params)

Khởi tạo: x0: [2.5, 2.5]
Chiếu lên C được: x: [2.5 2.5]


  0%|          | 0/10000 [00:00<?, ?it/s]


Hội tụ tại vòng lặp 0
+---+------------+---------+--------------+--------------+----------+----------+
| k | x_new      | gamma_k | y            | z_proj       |   e_x    |   e_f    |
+---+------------+---------+--------------+--------------+----------+----------+
| 0 | [2.5, 2.5] | 12.0000 | [0.25, 0.25] | [0.25, 0.25] | 0.000000 | 0.000000 |
+---+------------+---------+--------------+--------------+----------+----------+


   Dim 0: Min=0.0374, Max=0.8115
   Dim 1: Min=0.0374, Max=0.8115
-> Z* (Ideal): [0.0374 0.0374]
-> Ref Point (HV): [1.3927 1.3927]
-> x (Feasible): [2.5 2.5]


In [16]:
pf_true = []
converge = 0
for r in test_rays:
    x, y, conv = solve_weighted_chebyshev(
        r_pref=r,
        z_ref=z_star, 
        dim_x=dim_x, 
        dim_y=dim_y,
        params=params)
    if conv:
        converge += 1
    pf_true.append(f(x))
print(f"Converge rate = {100*converge/len(test_rays)}%")

r=[1. 0.]
Iter  | x          | y vector        | Objective  | Active Idx
0     | 2.4991     | [0.50, 0.58] | 0.212440     | f_1
2000  | 1.2163     | [0.50, 0.58] | 0.021794     | f_1
--> Hội tụ tại bước 2297!

r=[0.9995 0.0321]
Iter  | x          | y vector        | Objective  | Active Idx
0     | 2.4991     | [0.50, 0.58] | 0.212331     | f_1
2000  | 1.2167     | [0.50, 0.58] | 0.021827     | f_1
--> Hội tụ tại bước 2125!

r=[0.9979 0.0641]
Iter  | x          | y vector        | Objective  | Active Idx
0     | 2.4991     | [0.50, 0.58] | 0.212004     | f_1
--> Hội tụ tại bước 1777!

r=[0.9954 0.096 ]
Iter  | x          | y vector        | Objective  | Active Idx
0     | 2.4991     | [0.50, 0.58] | 0.211460     | f_1
--> Hội tụ tại bước 1543!

r=[0.9918 0.1279]
Iter  | x          | y vector        | Objective  | Active Idx
0     | 2.4991     | [0.50, 0.58] | 0.210698     | f_1
--> Hội tụ tại bước 1363!

r=[0.9872 0.1596]
Iter  | x          | y vector        | Objective  | Active Idx
0 

2000  | 3.7653     | [0.57, 0.51] | 0.023603     | f_2
--> Hội tụ tại bước 2350!

Converge rate = 100.0%


# Giải xấp xỉ dynamic

## Tuning

In [17]:
from tuning_dynamic import tuning_dynamic

In [ ]:
best_params, df = tuning_dynamic(
    prob, z_star, x_feasible, pf_true, ref_point, test_rays
)

=== GRID SEARCH: ƯU TIÊN MED ===
Số lượng cấu hình: 96 | Số tia: 50


0it [00:00, ?it/s]

!! Max_iter. Delta: 0.001780, Gap C: 0.000000, Gap Q: 0.022489
!! Max_iter. Delta: 0.000204, Gap C: 0.000000, Gap Q: 0.000023
!! Max_iter. Delta: 0.000395, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.000571, Gap C: 0.000000, Gap Q: 0.000000
-> Hội tụ sớm tại k=73.
!! Max_iter. Delta: 0.002691, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.002798, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.001207, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.002966, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.003031, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.001619, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.001746, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.001868, Gap C: 0.000000, Gap Q: 0.000000
-> Hội tụ sớm tại k=294.
!! Max_iter. Delta: 0.003200, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.003212, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.002305, Gap C: 0.000000, Gap Q: 0.000000
!! Max

1it [01:54, 114.11s/it]

!! Max_iter. Delta: 0.001777, Gap C: 0.000000, Gap Q: 0.023002
[00] MED: 3.764711e-03 | HV: 1.694546
!! Max_iter. Delta: 0.000002, Gap C: 0.000000, Gap Q: 0.017222
-> Hội tụ sớm tại k=271.
-> Hội tụ sớm tại k=649.
-> Hội tụ sớm tại k=911.
-> Hội tụ sớm tại k=73.
!! Max_iter. Delta: 0.002691, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.002798, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.001207, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.002966, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.003031, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.001619, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.001746, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.001868, Gap C: 0.000000, Gap Q: 0.000000
-> Hội tụ sớm tại k=294.
!! Max_iter. Delta: 0.003200, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.003212, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.002305, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta:

...

3it [05:46, 116.60s/it]

!! Max_iter. Delta: 0.003553, Gap C: 0.000000, Gap Q: 0.026454
[02] MED: 3.252324e-03 | HV: 1.700259
!! Max_iter. Delta: 0.003539, Gap C: 0.000000, Gap Q: 0.014554
-> Hội tụ sớm tại k=620.
!! Max_iter. Delta: 0.004472, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.001149, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.001489, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.005338, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.002119, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.002421, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.005966, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.006090, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.003244, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.006281, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.006354, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.006403, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.006437, Gap C: 0.000000, Gap Q: 0

4it [07:45, 117.60s/it]

!! Max_iter. Delta: 0.003539, Gap C: 0.000000, Gap Q: 0.014549
[03] MED: 3.932563e-03 | HV: 1.701113
!! Max_iter. Delta: 0.001847, Gap C: 0.000000, Gap Q: 0.003386
!! Max_iter. Delta: 0.000204, Gap C: 0.000000, Gap Q: 0.000023
!! Max_iter. Delta: 0.000395, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.000571, Gap C: 0.000000, Gap Q: 0.000000
-> Hội tụ sớm tại k=73.
!! Max_iter. Delta: 0.002691, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.002798, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.001207, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.002966, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.003031, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.001619, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.001746, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.001868, Gap C: 0.000000, Gap Q: 0.000000
-> Hội tụ sớm tại k=294.
!! Max_iter. Delta: 0.003200, Gap C: 0.000000, Gap Q: 0.000000
!! Max_iter. Delta: 0.003212, G

5it [09:38, 115.86s/it]

## Best

In [ ]:
from Phase2_dynamic import optim_Scalarization
from evaluate import get_metrics

In [ ]:
pf_true_dynamic = []
for r in test_rays:
        x_final, _ = optim_Scalarization(
            prob=prob,
            x_feasible=x_feasible,  
            r=r, 
            z_star=z_star,
            # Các tham số tĩnh
            verbose=True,
            # --- CÁC THAM SỐ ĐANG TỐI ƯU ---
            max_iter=best_params['max_iter'],
            mu=best_params['mu'],
            init_params=best_params['init_params'],
            expo_alpha=best_params['expo_alpha'],
            expo_lambda=best_params['expo_lambda'],
            expo_beta=best_params['expo_beta'],
            expo_gamma=best_params['expo_gamma'],
        )
        pf_true_dynamic.append(prob.objective_func(x_final))
    
med, hv = get_metrics(pf_true_dynamic, pf_true, prob, ref_point)

## Viz

In [ ]:
from utils import visualize_pareto_front, generate_pareto_grid

In [ ]:
def non_c(x):
    return 1
pf_cloud, pf_targets = generate_pareto_grid(
    f_func=f, 
    c_funcs=[non_c], 
    q_plus_func=q_plus, 
    resolution=500
)

In [ ]:
visualize_pareto_front(
    pf_pred=np.array(pf_true_dynamic), 
    pf_cloud=pf_cloud,   
    pf_targets=np.array(pf_true),
    title="Dynamic approximate exact",
    figsize=(8, 6)
)

# Hypernet

In [ ]:
from training_hypernet import train_hypernet

In [ ]:
prob_ = Problem(
    f=[f1, f2], jac_f=[jacobian_f],
    dim_x=dim_x, dim_y=dim_y,
    proj_C=proj_C_handler.project,
    proj_Qplus=proj_Q_handler.project
)

## Tuning

In [ ]:
from tuning_hypernet import run_hypernet_tuning

In [ ]:
param_grid = {
    'lr': [1e-3, 5e-4],
    'num_epochs': [500, 1000, 2000],
    
    'num_partitions': [10],
    
    # Tham số thuật toán 2-A: Tăng dần penalty
    'beta_C_0': [10],
    'beta_Q_0': [5],
    'rho_C': [1.03], 
    'rho_Q': [1.01, 1.03],
    
    # Cố định Max để tránh grid quá lớn 
    'beta_C_max': [1000.0],
    'beta_Q_max': [100.0, 500.0, 1000.0]
}

In [ ]:
device = 'cpu'

In [ ]:
results = run_hypernet_tuning(
    prob=prob_,
    dim_x=dim_x,
    z_star=z_star,
    ref_point=ref_point,
    test_rays=test_rays,
    pf_true=pf_true,
    param_grid=param_grid,
    indicator="MED", 
    device=device,
    save_dir=f"model/{case}",
    train_func=train_hypernet
)

# Viz

In [ ]:
import pandas as pd

In [ ]:
df = pd.json_normalize(results, sep='_')
df.columns = df.columns.str.replace('params_', '')

In [ ]:
df.sort_values("med").head(10)

In [ ]:
df.to_csv(f"exp/{case}/{n_rays}.csv", index=False)